# Bibliothek Installation

In [1]:
%pip install opencv-python pandas openpyxl pytesseract -U scikit-learn pymupdf matplotlib ipywidgets chardet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [ipywidgets]3 [ipywidgets]
Note: you may need to restart the kernel to use updated packages.


# Import von notwendigen Bibliotheken

In [2]:
import platform
import warnings
if platform.system() == 'Windows':
    print('Windows-System erkannt')
    warnings.warn(
        "Bei importproblemen Microsoft Visual C++ Redistributables installieren und die Anwendung neu starten\n"
        "https://learn.microsoft.com/en-us/cpp/windows/latest-supported-vc-redist?view=msvc-170\n",
        UserWarning
    )
import cv2
import pandas as pd
from pandas import DataFrame
from IPython.display import display,Markdown,HTML
import chardet
import time
import os
import csv
import tkinter as tk
from tkinter import filedialog,messagebox
from pathlib import Path
from matplotlib import pyplot as plt
import re
import sys
import fitz 
import difflib
import openpyxl
from datetime import datetime
import numpy as np
import pytesseract
from pytesseract import Output
from sklearn.cluster import KMeans
from openpyxl.styles import PatternFill

In [3]:
if platform.system() == 'Windows':
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
    print('Tesseract Path set for Windows!')
if platform.system() == "Linux" and os.name == "posix":
    pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'
    print('Tesseract Path set for Linux-Systems!')

In [4]:
# Custom Print Funktion
def printmd(*args):
    text = " ".join(str(arg) for arg in args)
    display(Markdown(f"<div style='word-wrap: break-word;'><h5>Log: {text}</h5></div>"))

# Custom Image Display Funktion
def display_images(*args):
    def get_image_path(image):
        return str(image) if isinstance(image, Path) else image

    if len(args) == 1 and isinstance(args[0], (str, Path)):
        image_path = get_image_path(args[0])  
        image_name = Path(image_path).name 
        
        display(HTML(f"<div style='text-align: center;'><h5>{image_name}</h5><img src='{image_path}' style='max-width: 100%; max-height: 400px;'></div>"))
    
    elif len(args) == 1 and isinstance(args[0], list):
        image_paths = args[0]
        image_tags = ""
        
        for image_path in image_paths:
            image_path = get_image_path(image_path)
            image_name = Path(image_path).name 
            image_tags += f"<div style='display: inline-block; text-align: center; margin: 10px;'><h5>{image_name}</h5><img src='{image_path}' style='max-width: 100%; max-height: 400px;'></div>"

        display(HTML(f"<div style='display: flex; justify-content: center; flex-wrap: wrap;'>{image_tags}</div>"))

### Code für die Grafische Oberfläche

In [5]:
%gui tk

# Variablen um Pfade abzuspeichern 
selected_folder = ""
selected_folder_GL = ""
gesamtliste_ba = None
gesamtliste_ma = None
attendance_lists = []
online_lists = []

# Funktion zum Suchen der Gesamtlisten
def find_gesamtlisten(root,file):
    global gesamtliste_ba, gesamtliste_ma
    if file.endswith(('.xlsx', '.csv')) and not (file.startswith('._')or file in ['.DS_Store','Thumbs.db','Desktop.ini']):  
        file_path = Path(os.path.join(root, file))
        if re.search(r"bachelor", file, re.IGNORECASE):
            gesamtliste_ba = file_path
        else: 
            gesamtliste_ma = file_path

# Funktion zum Suchen von Präsenzlisten
def find_attendance_lists(folder):
    global attendance_lists
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.startswith('._')or file in ['.DS_Store','Thumbs.db','Desktop.ini']:
                continue
            if file.lower().endswith(".pdf"):  
                attendance_lists.append(Path(root) / file)

# Funktion zum Suchen von Onlinelisten
def find_online_lists(folder):
    global online_lists
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.startswith('._')or file in ['.DS_Store','Thumbs.db','Desktop.ini']:
                continue
            if file.lower().endswith((".csv", ".xlsx", ".pdf")):  
                online_lists.append(Path(root) / file)

def clear_all_selections():
    global selected_folder, selected_folder_GL , gesamtliste_ba,gesamtliste_ma,attendance_lists,online_lists
    selected_folder = ""
    gesamtliste_ba = None
    gesamtliste_ma = None
    attendance_lists = []
    online_lists = []
    
    label.config(text="No folder selected")
    label_gl.config(text="No folder selected")

def select_folder():
    global selected_folder, selected_folder_GL
    clear_all_selections() 
    folder_path = filedialog.askdirectory(mustexist=True, title="Select T Folders", initialdir=os.getcwd())

    if folder_path:
            folder_name = os.path.basename(folder_path)
            # Prüfen ob gewählter ordner T1-T8 ist
            if folder_name.startswith("T") and folder_name[1:].isdigit() and 1 <= int(folder_name[1:]) <= 8:

                #Pfadzusammensetzung für T-Ordner in Ausgewertet
                ausgewertet_folder = os.path.join(os.path.dirname(folder_path), "ausgewertet", folder_name)
                
                if os.path.exists(ausgewertet_folder): 
                    messagebox.showinfo("Success", f"Selected {folder_name} and its corresponding folder in 'ausgewertet'.")
                    selected_folder= folder_path
                    selected_folder_GL = ausgewertet_folder
                    label.config(text=f"Selected T Folder: {selected_folder}")
                    label_gl.config(text=f"Selected T Folder in ausgewertet: {selected_folder_GL}")
                else:
                    messagebox.showwarning("Warning", f"Corresponding folder in 'ausgewertet' does not exist for {folder_name}.")
            else:
                messagebox.showwarning("Warning", "Please select a folder from T1 to T8.")
    else:
            messagebox.showwarning("Warning", "No folder selected.")

def auswerten():
    printmd("Auswerten button clicked")
    if selected_folder_GL is not None:
        for root, dirs, files in os.walk(selected_folder_GL):
            for file in files:
                find_gesamtlisten(root,file)
        for root, dirs, files in os.walk(selected_folder):
            online_folder = Path(os.path.join(root, "online"))
            find_attendance_lists(root)
            if online_folder.exists() and online_folder.is_dir():
                find_online_lists(online_folder)
    close_app()
        

def close_app():
    root.quit()
    root.destroy()

def close_app_withx():
    clear_all_selections()
    close_app()
# Main window
root = tk.Tk()
root.title("Automation Bot")

root.protocol("WM_DELETE_WINDOW", close_app_withx)

window_width = 800
window_height = 600
root.geometry(f"{window_width}x{window_height}")

# Buttons und Labels 
select_button = tk.Button(root, text="Ordner auswählen", command=select_folder)
select_button.pack(pady=10)

label = tk.Label(root, text="No folder selected")
label.pack(pady=5)

label_gl = tk.Label(root, text="No folder selected")
label_gl.pack(pady=5)

auswerten_button = tk.Button(root, text="Auswerten", command=auswerten)
auswerten_button.pack(pady=10)

root.mainloop()

#print(selected_folder,selected_folder_GL)
printmd(attendance_lists)
printmd(online_lists)

<div style='word-wrap: break-word;'><h5>Log: []</h5></div>

<div style='word-wrap: break-word;'><h5>Log: []</h5></div>

In [6]:
if not gesamtliste_ba or not gesamtliste_ma:
        printmd("Gesamtlisten nicht bereitgestellt!")
        sys.exit()

start_time = time.time()

<div style='word-wrap: break-word;'><h5>Log: Gesamtlisten nicht bereitgestellt!</h5></div>

SystemExit: 

/Users/aashiskaphle/Downloads/Anhang/app/.venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3678: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Umwandlung Pdf zu Bild

In [ ]:
#Für bearbeitung von Präsenzlisten

printmd('Pfade der Präsenzlisten:',attendance_lists)
base_output_folder = Path('logs')
base_output_folder.mkdir(parents=True, exist_ok=True)
images_for_OCR = {}
process_output_folder = None

def create_unique_output_folder(base_folder):
    # Generieruung eines Ordners für die Ablage von Outputs
    timestamp = None
    if platform.system() == 'Windows':
        timestamp = datetime.now().strftime('%Y_%m_%d_%H-%M-%S')
    else:
        timestamp = datetime.now().strftime('%Y_%m_%d_%H:%M:%S')
    unique_folder = base_folder / f"process_{timestamp}"
    unique_folder.mkdir(parents=True, exist_ok=True)
    return unique_folder

def preprocess_image_for_ocr(input_path, output_path):
    image = cv2.imread(input_path, cv2.IMREAD_COLOR)
    assert image is not None, "file could not be read, check with os.path.exists()"
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _,binary = cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    cv2.imwrite(output_path,binary)
    printmd('Bild verarbeitet')
    display_images([input_path,output_path])

def convert_pdf_to_images(pdf_path, output_folder, scale_factor):
    pdf_document = fitz.open(pdf_path)
    pdf_name = pdf_path.stem  
    images_for_OCR[str(pdf_name)] = images_for_OCR.get(str(pdf_name), [])
    matrix = fitz.Matrix(scale_factor, scale_factor)
    pdf_output_folder = output_folder / pdf_name
    pdf_output_folder.mkdir(parents=True, exist_ok=True)

    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)  
        pix = page.get_pixmap(matrix=matrix)  
        image_path = pdf_output_folder / f"page_{page_num + 1}.png"
        pix.save(image_path)  
        printmd(f"Converted PDF {pdf_name} to image {image_path} (Resolution: {pix.width}x{pix.height})")
        image_path_processed = pdf_output_folder / f"page_{page_num + 1}_processed.png"
        preprocess_image_for_ocr(image_path,image_path_processed)
        images_for_OCR[str(pdf_name)].append(str(image_path_processed))
        printmd(f"Saved Processed image to {image_path_processed}")
        
# Verarbeiten der Präsenzlisten
if attendance_lists:
    process_output_folder = create_unique_output_folder(base_output_folder)
    for pdf_path in attendance_lists:
        convert_pdf_to_images(pdf_path, process_output_folder, scale_factor=2.75) 


<div style='word-wrap: break-word;'><h5>Log: Pfade der Präsenzlisten: [PosixPath('/Users/aashiskaphle/Downloads/BA/Listenvorlagen/Anwesenheitslisten/WS_2024_2025/T1/Master_1_Sem_WPM-AdKI_Müller.pdf')]</h5></div>

<div style='word-wrap: break-word;'><h5>Log: Converted PDF Master_1_Sem_WPM-AdKI_Müller to image logs/process_2025_05_15_09:53:28/Master_1_Sem_WPM-AdKI_Müller/page_1.png (Resolution: 1637x2316)</h5></div>

<div style='word-wrap: break-word;'><h5>Log: Bild verarbeitet</h5></div>

<div style='word-wrap: break-word;'><h5>Log: Saved Processed image to logs/process_2025_05_15_09:53:28/Master_1_Sem_WPM-AdKI_Müller/page_1_processed.png</h5></div>

<div style='word-wrap: break-word;'><h5>Log: Converted PDF Master_1_Sem_WPM-AdKI_Müller to image logs/process_2025_05_15_09:53:28/Master_1_Sem_WPM-AdKI_Müller/page_2.png (Resolution: 1637x2316)</h5></div>

<div style='word-wrap: break-word;'><h5>Log: Bild verarbeitet</h5></div>

<div style='word-wrap: break-word;'><h5>Log: Saved Processed image to logs/process_2025_05_15_09:53:28/Master_1_Sem_WPM-AdKI_Müller/page_2_processed.png</h5></div>

## DataFrame selection of rows and columns with iloc Method
* `df.iloc[0, 1]`: Gets the value in the first row, second column
* `df.iloc[:, 1]`: Gets the entire second column.
* `df.iloc[1, :]`: Gets the entire second row.
* `df.iloc[:, 0]`: Selects all rows from the first column of the DataFrame.
* `matching_indices = df[df.iloc[:, 0].str.contains('name', na=False)].index[0]`: Getting index in the first column where the cell contains 'name'

### OCR Durchführung und Umwanndlung der Dataframes zu anwesenheitstabellen

In [ ]:
from helperfunctions import draw_contours,improve_ocr_result,recheck_output_table
from imagetotable import convert_ocrdata_to_table
final_attendance_tables = {}

if images_for_OCR:
    for key,values in images_for_OCR.items():
        printmd(f"-------- Verabeitung der Liste {key} gestartet: --------")
        final_attendance_tables[str(key)] = final_attendance_tables.get(str(key), [])
        for value in values:
            directory = os.path.dirname(value)
            csv_path = directory + "/csv_exports"
            os.makedirs(csv_path, exist_ok=True)
            filename = os.path.splitext(os.path.basename(value))[0]
            printmd('Processing Directory:',directory, 'Image:', value)
            image = cv2.imread(value,cv2.IMREAD_GRAYSCALE)
            display_images(value)
            al = pytesseract.image_to_data(image,output_type=Output.DATAFRAME, lang='deu')
            printmd('OCR DataFrame')
            display(al)
            al.to_csv(f"{csv_path}/{filename}_df.csv", index=False)
            rgb_img = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
            contours_in_image = draw_contours(rgb_img,al)
            con_imgpath = f"{directory}/{filename}_contours.png"
            cv2.imwrite(con_imgpath,contours_in_image)
            printmd('OCR Detected areas')
            display_images(con_imgpath)
            improved_ocr_path = directory + '/ocr_improvements'+ f"/{filename}"
            os.makedirs(improved_ocr_path, exist_ok=True)
            al = improve_ocr_result(al,image,improved_ocr_path)
            file_list = [os.path.join(improved_ocr_path, f) for f in os.listdir(improved_ocr_path)]
            printmd(f" -------- Verbesserung der OCR Erkennung für Folgender Bereiche --------")
            display_images(file_list)
            al = convert_ocrdata_to_table(al)
            printmd('Converted to Original Table')
            recheck_output_table(al)
            printmd('Rechecking Table ...')
            display(al)
            saving_path = f"{csv_path}/{filename}_final_table.csv"
            al.to_csv(saving_path, index=False)
            final_attendance_tables[str(key)].append(str(saving_path))
            printmd(f" -------- Verarbeitung abgeschlossen für Bild {value} --------")
    printmd(f" -------- Verarbeitung abgeschlossen für Modul {key} --------")

printmd(final_attendance_tables)

<div style='word-wrap: break-word;'><h5>Log: -------- Verabeitung der Liste Master_1_Sem_WPM-AdKI_Müller gestartet: --------</h5></div>

<div style='word-wrap: break-word;'><h5>Log: Processing Directory: logs/process_2025_05_15_09:53:28/Master_1_Sem_WPM-AdKI_Müller Image: logs/process_2025_05_15_09:53:28/Master_1_Sem_WPM-AdKI_Müller/page_1_processed.png</h5></div>

<div style='word-wrap: break-word;'><h5>Log: OCR DataFrame</h5></div>

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
0,1,1,0,0,0,0,0,0,1637,2316,-1.0,NaN
1,2,1,1,0,0,0,145,117,1347,1,-1.0,NaN
2,3,1,1,1,0,0,145,117,1347,1,-1.0,NaN
3,4,1,1,1,1,0,145,117,1347,1,-1.0,NaN
4,5,1,1,1,1,1,145,117,1347,1,95.0,
...,...,...,...,...,...,...,...,...,...,...,...,...
474,5,1,94,1,1,1,145,2103,1347,1,95.0,
475,2,1,95,0,0,0,1489,117,4,1987,-1.0,NaN
476,3,1,95,1,0,0,1489,117,4,1987,-1.0,NaN
477,4,1,95,1,1,0,1489,117,4,1987,-1.0,NaN


<div style='word-wrap: break-word;'><h5>Log: OCR Detected areas</h5></div>

...

<div style='word-wrap: break-word;'><h5>Log: Rechecking Table ...</h5></div>

,0,1,2,3
0,None,"Name, Vorname (Druckschrift!)",Unterschrift:,Matr.-Nr. oder
1,None,None,None,FH-zugehörigkeit
2,29,Patrick Voigt,None,None
3,30,Layla Al-Mansour,None,None
4,32,Nour Abdulwahid,rent,None
5,34 Amira Etemadi,Daniel Ludwig,abs Koknann,None
6,35,Ralf Seidel,az,None
7,36,Faris Haddadi,Zu,None
8,8.,Jonas Maier,ulm,None
9,38,Hiba Karam,None,None


<div style='word-wrap: break-word;'><h5>Log:  -------- Verarbeitung abgeschlossen für Bild logs/process_2025_05_15_09:53:28/Master_1_Sem_WPM-AdKI_Müller/page_2_processed.png --------</h5></div>

<div style='word-wrap: break-word;'><h5>Log:  -------- Verarbeitung abgeschlossen für Modul Master_1_Sem_WPM-AdKI_Müller --------</h5></div>

<div style='word-wrap: break-word;'><h5>Log: {'Master_1_Sem_WPM-AdKI_Müller': ['logs/process_2025_05_15_09:53:28/Master_1_Sem_WPM-AdKI_Müller/csv_exports/page_1_processed_final_table.csv', 'logs/process_2025_05_15_09:53:28/Master_1_Sem_WPM-AdKI_Müller/csv_exports/page_2_processed_final_table.csv']}</h5></div>

#### Vorlesungstage/Datei- & Tabellenabkürzungen BA & MA

In [ ]:
lecture_dates = {
    "T1": "2024-09-28",
    "T2": "2024-10-12",
    "T3": "2024-11-09",
    "T4": "2024-11-23",
    "T5": "2024-11-30",
    "T6": "2024-12-14",
    "T7": "2025-01-11",
    "T8": "2025.01-25"
}
ma_filesheet_names = {
    "IT-Controlling": "IT-Controlling",
    "IuK": "IuK",
    "IT-Sicherheit": "IT-Sicherheit",
    "UsaEng": "Usability Engineering",
    "IT-Management": "IT-Management",
    "IT-Consulting": "IT-Consulting",
    "RegEng": "Requirements Engineering f.d.L.",
    "WPM-IntDB": "InternetDB",
    "WPM-AdKI": "Anwendungen der KI",
    "WPM-KoSy": "Kooperative Systeme"
}
ba_filesheet_names = {
    "EWI": "EWI",
    "AuP I": "AuP_I",
    "MA I": "Mathe I",
    "AuP II": "AuP II",
    "MA II": "Mathe II",
    "BWL I": "BWL_I",
    "WiMa": "Wima",
    "SPR": "SPR",
    "InfGr": "Informatik Grundlagen",
    "RE": "RE",
    "PM": "PM",
    "BWL II": "BWL_II",
    "DuI": "DuI",
    "VWL": "VWL",
    "BR": "Betr. Rechnungswesen",
    "BSWE": "BSWE",
    "GPM": "GPM",
    "BAWS": "Betr. Anwendungssysteme",
    "MoKo": "Mobile Kommunikation",
    "GdKI": "Grundlagen der KI"
}
printmd('Ausgewählter Termin Pfad:',selected_folder)
printmd('Ausgewählter T-Ordner: ',os.path.basename(selected_folder))

<div style='word-wrap: break-word;'><h5>Log: Ausgewählter Termin Pfad: /Users/aashiskaphle/Downloads/BA/Listenvorlagen/Anwesenheitslisten/WS_2024_2025/T1</h5></div>

<div style='word-wrap: break-word;'><h5>Log: Ausgewählter T-Ordner:  T1</h5></div>

### Suche und Eintragung der Anwesenheiten für Präsenzlisten

In [ ]:
from savingfunctions import save_attendance

if final_attendance_tables:
    print(final_attendance_tables)
    list_to_process = None
    lecture_date = None
    table_to_process = None
    sheet_name = None
    process_list_sheet = None
    current_t_folder = os.path.basename(selected_folder).upper()
    lecture_date = lecture_dates[current_t_folder]

    for key,values in final_attendance_tables.items():
        if 'bachelor' in key.lower():
            list_to_process = gesamtliste_ba
            gl = pd.ExcelFile(list_to_process)
            sheet_names = gl.sheet_names
            file_shortcut = key.split("_")
            file_shortcut = file_shortcut[3]
            printmd('Datei abkürzung: ',file_shortcut)
            printmd('In Excel enthaltene Tabellen: ',sheet_names)
            for key1,_ in ba_filesheet_names.items():
                if key1.lower() == str(file_shortcut).lower():
                    sheet_name = ba_filesheet_names[key1]
                    break
            process_list_sheet = pd.read_excel(list_to_process, sheet_name=sheet_name)
            printmd('Bachelor Modul gefunden ','Date:',lecture_date)

        elif 'master' in key.lower():
            list_to_process = gesamtliste_ma
            gl = pd.ExcelFile(list_to_process)
            sheet_names = gl.sheet_names
            file_shortcut = key.split("_")
            file_shortcut = file_shortcut[3]
            printmd('Excel Sheets names:',sheet_names)
            for key1,_ in ma_filesheet_names.items():
                if key1.lower() == str(file_shortcut).lower():
                    sheet_name = ma_filesheet_names[key1]
                    break
            process_list_sheet = pd.read_excel(list_to_process, sheet_name=sheet_name)
            printmd('Master Modul gefunden! Datum des Termins: ',lecture_date)

        else:
            continue
        
        for value in values:
            table_to_process = pd.read_csv(value)
            save_attendance(lecture_date,process_list_sheet,table_to_process,list_to_process,sheet_name)
            table_to_process = None

printmd(f"Total processing time: {time.time() - start_time:.2f} seconds")

{'Master_1_Sem_WPM-AdKI_Müller': ['logs/process_2025_05_15_09:53:28/Master_1_Sem_WPM-AdKI_Müller/csv_exports/page_1_processed_final_table.csv', 'logs/process_2025_05_15_09:53:28/Master_1_Sem_WPM-AdKI_Müller/csv_exports/page_2_processed_final_table.csv']}


<div style='word-wrap: break-word;'><h5>Log: Excel Sheets names: ['IuK', 'IT-Controlling', 'IT-Management', 'InternetDB', 'Kooperative Systeme', 'Anwendungen der KI']</h5></div>

<div style='word-wrap: break-word;'><h5>Log: Master Modul gefunden! Datum des Termins:  2024-09-28</h5></div>

,0,1,2,3
0,NaN,Verbundstudiengang Wirtschaftsinformatik,NaN,NaN
1,NaN,"Anwesenheit in Gummersbach, anschließend",ins Postfach 180 zurück!,NaN
2,Datum:,Lehrender:,NaN,Raum:
3,Fach:,NaN,Uhrzeit/Gruppe:,NaN
4,"Name, Vorname (Druckschrift!)",NaN,Unterschrift:,Matr.-Nr. oder
5,NaN,NaN,NaN,FH-zugehörigkeit
6,2.,Nico Wagner,"or” Pe ma ww MA ualz we Dr Be a fer, A Um Sulz...",NaN
7,3.,o ol A an m,NaN,NaN
8,4.,David Klein,NaN,NaN
9,5.,Ali Hassan,NaN,NaN


,Verbundstudiengang Wirtschaftsinformatik,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
1,Fach,Anwendungen der KI in der WI,Lehrende,Bab,NaN,NaT,NaT,NaT,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
3,Name,Vorname,DO-Ma.Nr.,GM-Ma.Nr.,NaN,NaT,NaT,NaT,Anzahl der Termine,4.0
4,NaN,NaN,NaN,NaN,2024-09-28 00:00:00,2024-10-12,2024-11-09,2024-11-23,%-Anwesenheit,NaN
...,...,...,...,...,...,...,...,...,...,...
57,Mina,Salıh,NaN,NaN,x,NaT,NaT,NaT,NaN,NaN
58,Basma,Fadel,NaN,NaN,x,NaT,NaT,NaT,NaN,NaN
59,Julian,Dietrich,NaN,NaN,x,NaT,NaT,NaT,NaN,NaN
60,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN


Student gefunden:  Gesamtliste: Nico, Wagner Anwesenheitsliste: Nico Wagner
Anwesenheit ist bereits eingetragen für: Nico Wagner


,0,1,2,3
0,NaN,"Name, Vorname (Druckschrift!)",Unterschrift:,Matr.-Nr. oder
1,NaN,NaN,NaN,FH-zugehörigkeit
2,29,Patrick Voigt,NaN,NaN
3,30,Layla Al-Mansour,NaN,NaN
4,32,Nour Abdulwahid,rent,NaN
5,34 Amira Etemadi,Daniel Ludwig,abs Koknann,NaN
6,35,Ralf Seidel,az,NaN
7,36,Faris Haddadi,Zu,NaN
8,8.,Jonas Maier,ulm,NaN
9,38,Hiba Karam,NaN,NaN


,Verbundstudiengang Wirtschaftsinformatik,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
1,Fach,Anwendungen der KI in der WI,Lehrende,Bab,NaN,NaT,NaT,NaT,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
3,Name,Vorname,DO-Ma.Nr.,GM-Ma.Nr.,NaN,NaT,NaT,NaT,Anzahl der Termine,4.0
4,NaN,NaN,NaN,NaN,2024-09-28 00:00:00,2024-10-12,2024-11-09,2024-11-23,%-Anwesenheit,NaN
...,...,...,...,...,...,...,...,...,...,...
57,Mina,Salıh,NaN,NaN,x,NaT,NaT,NaT,NaN,NaN
58,Basma,Fadel,NaN,NaN,x,NaT,NaT,NaT,NaN,NaN
59,Julian,Dietrich,NaN,NaN,x,NaT,NaT,NaT,NaN,NaN
60,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN


Student gefunden:  Gesamtliste: Nour, Abdulwahid Anwesenheitsliste: Nour Abdulwahid
Anwesenheit ist bereits eingetragen für: Nour Abdulwahid
Student gefunden:  Gesamtliste: Daniel, Ludwig Anwesenheitsliste: Daniel Ludwig
Anwesenheit ist bereits eingetragen für: Daniel Ludwig
Student gefunden:  Gesamtliste: Ralf, Seidel Anwesenheitsliste: Ralf Seidel
Anwesenheit ist bereits eingetragen für: Ralf Seidel
Student gefunden:  Gesamtliste: Faris, Haddadi Anwesenheitsliste: Faris Haddadi
Anwesenheit ist bereits eingetragen für: Faris Haddadi
Student gefunden:  Gesamtliste: Jonas, Maier Anwesenheitsliste:  Jonas Maier
Anwesenheit ist bereits eingetragen für:  Jonas Maier
Student gefunden:  Gesamtliste: Marc, Günther Anwesenheitsliste: Marc Günther
Anwesenheit ist bereits eingetragen für: Marc Günther
Student gefunden:  Gesamtliste: Oliver, Vogel Anwesenheitsliste: Oliver Vogel
Anwesenheit ist bereits eingetragen für: Oliver Vogel
Student gefunden:  Gesamtliste: Yassir, Darwish Anwesenheitsliste

<div style='word-wrap: break-word;'><h5>Log: Total processing time: 15.83 seconds</h5></div>

### Bereinigung der Onlinelisten

In [ ]:
print('Pfade der Onlinelisten:',online_lists)

from online import clean_meeting_data

online_lists_paths = {}
if online_lists:
    if process_output_folder:
        online_output_folder = str(process_output_folder) + "/online"
        os.makedirs(online_output_folder, exist_ok=True)
    else:
        process_output_folder = create_unique_output_folder(base_output_folder)
        online_output_folder = str(process_output_folder) + "/online"
        os.makedirs(online_output_folder, exist_ok=True)
    for path in online_lists:
        online_list_name = path.stem
        online_lists_paths[str(online_list_name)] = online_lists_paths.get(str(online_list_name),[])
        if os.path.splitext(path)[1].lower() == '.csv' or '.xlsx':
            online_output_path = online_output_folder + f"/{str(online_list_name)}"
            os.makedirs(online_output_path, exist_ok=True)
            online_csvexport_path = online_output_path + '/csv_exports'
            os.makedirs(online_csvexport_path, exist_ok=True)
            extracted_names = clean_meeting_data(path)
            saving_path_online = f"{online_csvexport_path}/{online_list_name}_final_table.csv"
            extracted_names.to_csv(saving_path_online,index=False)
            online_lists_paths[str(online_list_name)].append(str(saving_path_online))
        if os.path.splitext(path)[1].lower() == '.pdf':
            printmd(path)
printmd(online_lists_paths)


Pfade der Onlinelisten: [PosixPath('/Users/aashiskaphle/Downloads/BA/Listenvorlagen/Anwesenheitslisten/WS_2024_2025/T1/online/Bachelor_1_Sem_EWI_Haake_Gruppe_A.xlsx')]


#### --- **Zoom Data** ---

,Meeting-ID,Thema,Startzeit,Endzeit,Benutzer-E-Mail:,Dauer (Minuten),Teilnehmer
0,3443268643,Persönlicher Meetingraum von Conny Günthner,2023-11-04 08:50:22,2023-11-04 11:56:57,conny.guenthner@th-koeln.de,187,NaN
2,Name (Originalname),Benutzer-E-Mail:,Beitrittszeit,Beendigungszeit,Dauer (Minuten),Gast,Im Warteraum
3,Conny Günthner,conny.guenthner@th-koeln.de,2023-11-04 08:50:22,2023-11-04 11:56:57,187,Nein,Nein
4,Felix Braun,NaN,2023-11-04 09:00:04,2023-11-04 11:56:55,177,Ja,Nein
5,Felix Braun,NaN,2023-11-04 09:00:04,2023-11-04 11:56:55,177,Ja,Nein
6,Samira Chahrour,NaN,2023-11-04 09:03:53,2023-11-04 11:56:56,174,Ja,Nein
7,Omar Haddad,NaN,2023-11-04 09:00:05,2023-11-04 11:56:57,177,Ja,Nein
8,Emre Kara,NaN,2023-11-04 09:02:36,2023-11-04 09:03:32,1,Ja,Ja
9,Mustafa Arslan,NaN,2023-11-04 09:00:43,2023-11-04 09:01:05,1,Ja,Ja
10,Esra Ceylan,NaN,2023-11-04 09:00:07,2023-11-04 09:48:26,49,Ja,Nein


#### --- **Zoom Data cleaned** ---

2,Name (Originalname),Benutzer-E-Mail:,Beitrittszeit,Beendigungszeit,Dauer (Minuten),Gast,Im Warteraum
2,Felix Braun,NaN,2023-11-04 09:00:04,2023-11-04 11:56:55,177,Ja,Nein
3,Samira Chahrour,NaN,2023-11-04 09:03:53,2023-11-04 11:56:56,174,Ja,Nein
4,Omar Haddad,NaN,2023-11-04 09:00:05,2023-11-04 11:56:57,177,Ja,Nein
5,Emre Kara,NaN,2023-11-04 09:02:36,2023-11-04 09:03:32,1,Ja,Ja
6,Mustafa Arslan,NaN,2023-11-04 09:00:43,2023-11-04 09:01:05,1,Ja,Ja
7,Esra Ceylan,NaN,2023-11-04 09:00:07,2023-11-04 09:48:26,49,Ja,Nein
8,Katharina Busch,NaN,2023-11-04 09:03:32,2023-11-04 11:56:56,174,Ja,Nein
9,Tom Fischer,NaN,2023-11-04 08:57:05,2023-11-04 09:00:05,3,Ja,Ja
10,Sabine Krause,NaN,2023-11-04 09:03:24,2023-11-04 09:03:29,1,Ja,Ja
11,Aylin Demir,NaN,2023-11-04 08:50:44,2023-11-04 09:00:04,10,Ja,Ja


,Name (Originalname)
2,Felix Braun
3,Samira Chahrour
4,Omar Haddad
5,Emre Kara
6,Mustafa Arslan
7,Esra Ceylan
8,Katharina Busch
9,Tom Fischer
10,Sabine Krause
11,Aylin Demir


<div style='word-wrap: break-word;'><h5>Log: {'Bachelor_1_Sem_EWI_Haake_Gruppe_A': ['logs/process_2025_05_15_09:53:28/online/Bachelor_1_Sem_EWI_Haake_Gruppe_A/csv_exports/Bachelor_1_Sem_EWI_Haake_Gruppe_A_final_table.csv']}</h5></div>

### Übertragung der Online Anwesenheiten

In [ ]:
from savingfunctions import save_online_attendance
if online_lists_paths:
        list_to_process = None
        lecture_date = None
        table_to_process = None
        sheet_name = None
        process_list_sheet = None
        current_t_folder = os.path.basename(selected_folder).upper()

        for key,values in online_lists_paths.items():
            if 'bachelor' in key.lower():
                list_to_process = gesamtliste_ba
                gl = pd.ExcelFile(list_to_process)
                sheet_names = gl.sheet_names
                file_shortcut = key.split("_")
                file_shortcut = file_shortcut[3]
                printmd('file shortcut:',file_shortcut)
                printmd('Excel Sheets names:',sheet_names)
                for key1,_ in ba_filesheet_names.items():
                    if key1.lower() == str(file_shortcut).lower():
                        sheet_name = ba_filesheet_names[key1]
                        break
                process_list_sheet = pd.read_excel(list_to_process, sheet_name=sheet_name)
                lecture_date = lecture_dates[current_t_folder]
                printmd('Bachelor Modul gefunden','Date:',lecture_date)

            elif 'master' in key.lower():
                list_to_process = gesamtliste_ma
                gl = pd.ExcelFile(list_to_process)
                sheet_names = gl.sheet_names
                file_shortcut = key.split("_")
                file_shortcut = file_shortcut[3]
                printmd('Excel Sheets names:',sheet_names)
                for key1,_ in ma_filesheet_names.items():
                    if key1.lower() == str(file_shortcut).lower():
                        sheet_name = ma_filesheet_names[key1]
                        break
                process_list_sheet = pd.read_excel(list_to_process, sheet_name=sheet_name)
                lecture_date = lecture_dates[current_t_folder]
                printmd('Master Modul gefunden''Date:',lecture_date)

            else:
                continue
            
            for value in values:
                table_to_process = pd.read_csv(value)
                save_online_attendance(lecture_date,process_list_sheet,table_to_process,list_to_process,sheet_name)
                table_to_process = None

printmd(f"Total processing time: {time.time() - start_time:.2f} seconds")

<div style='word-wrap: break-word;'><h5>Log: file shortcut: EWI</h5></div>

<div style='word-wrap: break-word;'><h5>Log: Excel Sheets names: ['AuP_I', 'EWI', 'Mathe I', 'BWL_I', 'SPR', 'Wima', 'RE', 'PM', 'BWL_II', 'BSWE', 'GPM']</h5></div>

<div style='word-wrap: break-word;'><h5>Log: Bachelor Modul gefunden Date: 2024-09-28</h5></div>

,Name (Originalname)
0,Felix Braun
1,Samira Chahrour
2,Omar Haddad
3,Emre Kara
4,Mustafa Arslan
5,Esra Ceylan
6,Katharina Busch
7,Tom Fischer
8,Sabine Krause
9,Aylin Demir


,Verbundstudiengang Wirtschaftsinformatik,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
1,Fach,EWI,Lehrende,Vaziri,NaN,NaT,NaT,NaT,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
3,Name,Vorname,DO-Ma.Nr.,GM-Ma.Nr.,Termine,NaT,NaT,NaT,Anzahl der Termine,4.0
4,NaN,NaN,NaN,NaN,2024-09-28 00:00:00,2024-10-12,2024-11-09,2024-11-23,%-Anwesenheit,NaN
5,Jonas,Meyer,NaN,NaN,x,NaT,NaT,NaT,NaN,NaN
6,Aylin,Demir,NaN,NaN,x,NaT,NaT,NaT,NaN,NaN
7,Leon,Schneider,NaN,NaN,x,NaT,NaT,NaT,NaN,NaN
8,Fatima,Al-Mansour,NaN,NaN,x,NaT,NaT,NaT,NaN,NaN
9,Lukas,Becker,NaN,NaN,x,NaT,NaT,NaT,NaN,NaN


Felix Braun  wird ausgewertet...
date column: 4
Freie Zeilen: [0, 51, 52]
Student gefunden:  Gesamtliste: Felix, Braun Anwesenheitsliste: Felix Braun
row student: 21
Anwesenheit ist bereits eingetragen für: Felix Braun
Samira Chahrour  wird ausgewertet...
date column: 4
Freie Zeilen: [0, 51, 52]
Student gefunden:  Gesamtliste: Samira, Chahrour Anwesenheitsliste: Samira Chahrour
row student: 53
Anwesenheit ist bereits eingetragen für: Samira Chahrour
Omar Haddad  wird ausgewertet...
date column: 4
Freie Zeilen: [0, 51, 52]
Student gefunden:  Gesamtliste: Omar, Haddad Anwesenheitsliste: Omar Haddad
row student: 24
Anwesenheit ist bereits eingetragen für: Omar Haddad
Emre Kara  wird ausgewertet...
date column: 4
Freie Zeilen: [0, 51, 52]
Student gefunden:  Gesamtliste: Emre, Kara Anwesenheitsliste: Emre Kara
row student: 47
Anwesenheit ist bereits eingetragen für: Emre Kara
Mustafa Arslan  wird ausgewertet...
date column: 4
Freie Zeilen: [0, 51, 52]
Student gefunden:  Gesamtliste: Mustafa

<div style='word-wrap: break-word;'><h5>Log: Total processing time: 25.69 seconds</h5></div>